# Training Data

<img src="Images/lab5_1.png">

<img src="Images/lab5_2.png">

In [1]:
import tensorflow as tf

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]] #인풋
y_data = [[0], [0], [0], [1], [1], [1]] #정답 레이블(True and False, Pass or Fail)

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 2]) #shape=[None, 2] (개수 제한 없음(None) : x의 전체 데이터 개수는 n개, 각 요소는 2개의 값을 가진다)
Y = tf.placeholder(tf.float32, shape=[None, 1]) #shape=[None, 1] (개수 제한 없음(None) : y의 전체 데이터 개수는 n개, 각 요소는 1개의 값을 가진다)
W = tf.Variable(tf.random_normal([2, 1]), name='weight') #들어오는 값 2 : X(인풋 수), 나가는 값 1 : Y(정답레이블, 아웃풋)
b = tf.Variable(tf.random_normal([1]), name='bias') #들어오는 값 1 : 편향은 항상 나가는 값과 같다.

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W) + b))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b) #인풋과 가중치를 내적 곱 한 후에 편향을 더한다. 활성함수는 시그모이드
#sigmoid = 1 / 1 + e^-x

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost) #체인룰로 최적화까지 같이

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32) 
#활성함수(시그모이드) 거친 값이 0.5보다 크면 Pass. 
#float32로 캐스팅 하면 True이면 1, False이면 0
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
#정확도 계산 예측값(predicted와 Y값이 일치하면(equl) 1, 일치하지 않으면 0. 그것의 평균을 구한다.

# Launch graph
with tf.Session() as session:
    # Initialize TensorFlow variables
    session.run(tf.global_variables_initializer()) #초기화

    for step in range(10001):
        cost_val, _ = session.run([cost, train], feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, cost_val)

    # Accuracy report
    h, c, a = session.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)




0 1.20933
200 0.496147
400 0.451764
600 0.424833
800 0.405326
1000 0.389458
1200 0.375628
1400 0.363096
1600 0.351492
1800 0.340618
2000 0.330358
2200 0.320639
2400 0.311412
2600 0.302638
2800 0.294285
3000 0.286328
3200 0.278742
3400 0.271506
3600 0.264599
3800 0.258003
4000 0.2517
4200 0.245673
4400 0.239907
4600 0.234388
4800 0.2291
5000 0.224033
5200 0.219172
5400 0.214509
5600 0.21003
5800 0.205728
6000 0.201591
6200 0.197613
6400 0.193783
6600 0.190095
6800 0.186542
7000 0.183116
7200 0.179812
7400 0.176623
7600 0.173543
7800 0.170568
8000 0.167692
8200 0.16491
8400 0.162219
8600 0.159614
8800 0.157091
9000 0.154647
9200 0.152278
9400 0.14998
9600 0.147751
9800 0.145588
10000 0.143487

Hypothesis:  [[ 0.02825372]
 [ 0.15533394]
 [ 0.29273912]
 [ 0.78697848]
 [ 0.94304717]
 [ 0.98134047]] 
Correct (Y):  [[ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]] 
Accuracy:  1.0


# Classifying diabetes

In [2]:
import numpy as np

xy = np.loadtxt("data-03-diabetes.csv", delimiter=',', dtype=np.float32) #파일에서 불러온다.
x_data = xy[:, 0:-1] #전체 행을 가져온다, 각 행의 0 ~ index-1 행까지 가져온다.
y_data = xy[:, [-1]] #전체 행을 가져온다, 각 행의 마지막 행만 가져온다.

print(x_data.shape, y_data.shape)

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 8]) #전체 요소의 개수는 n, 각 요소는 8개씩
Y = tf.placeholder(tf.float32, shape=[None, 1]) #전체 요소의 개수는 n, 각 요소는 1개씩

W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')
#tf.placeholder()는 입력 데이터를 만들 때 주로 사용한다. (실제 훈련 예제를 제공하는 변수) - 초기값을 지정할 필요 없다. (모델 입력시 변경되지 않을 데이터)
#tf.Variable()은 데이터의 상태를 저장할 때 주로 사용한다. (가중치나 편향 등의 학습 가능한 변수) - 초기값을 지정해야 한다. (학습 되는 데이터)
#http://stackoverflow.com/questions/36693740/whats-the-difference-between-tf-placeholder-and-tf-variable

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b) #행렬 곱

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis)) #코스트 함수

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost) #최적화.

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32) #0.5보다 크면 Pass
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, cost_val)

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)


(759, 8) (759, 1)
0 0.916749
200 0.74846
400 0.682445
600 0.653924
800 0.635843
1000 0.621249
1200 0.608553
1400 0.5973
1600 0.587271
1800 0.578306
2000 0.570272
2200 0.563054
2400 0.556556
2600 0.55069
2800 0.545384
3000 0.540571
3200 0.536197
3400 0.532211
3600 0.528572
3800 0.525243
4000 0.522189
4200 0.519383
4400 0.5168
4600 0.514416
4800 0.512213
5000 0.510172
5200 0.508279
5400 0.50652
5600 0.504882
5800 0.503356
6000 0.50193
6200 0.500596
6400 0.499347
6600 0.498176
6800 0.497076
7000 0.496042
7200 0.495068
7400 0.49415
7600 0.493284
7800 0.492466
8000 0.491692
8200 0.490959
8400 0.490265
8600 0.489606
8800 0.488981
9000 0.488387
9200 0.487822
9400 0.487284
9600 0.486772
9800 0.486283
10000 0.485817

Hypothesis:  [[ 0.47367424]
 [ 0.9413963 ]
 [ 0.31415501]
 [ 0.94134706]
 [ 0.3666085 ]
 [ 0.76241398]
 [ 0.94528311]
 [ 0.6179769 ]
 [ 0.29010585]
 [ 0.44373426]
 [ 0.59621358]
 [ 0.1788163 ]
 [ 0.36583996]
 [ 0.48122874]
 [ 0.79059917]
 [ 0.35119659]
 [ 0.72932601]
 [ 0.87499112]